# Fine-Tuning GPT-2 + LoRA for COVID-19 Vaccine Misinformation Correction
This notebook demonstratesthe step where we fine-tune a pre-trained GPT-2 model using **LoRA** on the **Vax-Culture dataset**  to generate **factually correct explanations** for COVID-19 vaccine-related misinformation.


In [ ]:
#install packages
!pip install -U transformers datasets peft accelerate evaluate
#imports

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd
import numpy as np



## Download & Load model

In [ ]:
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# i removed this cell output due to a widget error

## Prompt Hinting
During training, we include the binary labels as prompt hints to help the model learn which aspects of the tweet to focus on


In [ ]:
binary_cols = [col for col in df.columns if col.startswith('criticism_') or col.startswith('support_')]

def create_prompt(row):
    labels_str = ", ".join([f"{col}={row[col]}" for col in binary_cols])
    prompt = f"Tweet: {row['tweet_text']}\nLabels: {labels_str}\nCorrected explanation:"
    target = f"{row['communicated_message']}: {row['tweet_text']}"
    return {"input_text": prompt, "target_text": target}

dataset_dict = df.apply(create_prompt, axis=1).tolist()
dataset = Dataset.from_list(dataset_dict)

## Train/test split

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
import json
import os

train_list = train_dataset.to_dict() if hasattr(train_dataset, "to_dict") else train_dataset
test_list = test_dataset.to_dict() if hasattr(test_dataset, "to_dict") else test_dataset

output_dir = "/content/drive/MyDrive/dataset/"
os.makedirs(output_dir, exist_ok=True)

with open(os.path.join(output_dir, "train.json"), "w") as f:
    json.dump(train_list, f, indent=4)

with open(os.path.join(output_dir, "test.json"), "w") as f:
    json.dump(test_list, f, indent=4)

## Tokenization


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    return tokenizer(batch["input_text"], truncation=True, padding="max_length", max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
# i removed this cell output due to a widget error

## Applying LoRA (Low-Rank Adaptation)
To fine tune efficiently by updating only a small subset of model parameters.

- We wrap LLaMA with LoRA adapters on the attention layers (q_proj and v_proj), so only the small LoRA parameters are trained while the original model weights stay frozen.
- The q_proj and v_proj layers are part of the attention mechanism, which captures most of the model’s knowledge, so adding LoRA adapters here lets the model efficiently learn task-specific adjustments without updating all weights.

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

## Training arguments

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/gpt2_finetuned_vax",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=200,
    logging_steps=20,
    learning_rate=1.5e-4,
    num_train_epochs=2,
    weight_decay=0.01,
    warmup_steps=30,
    fp16=True,
    save_total_limit=3,
    load_best_model_at_end=True,
    report_to=[]
)

## Training

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,

)
trainer.train()

During training, both the training and validation losses gradually decreased, with final values around 0.486, showing that the model learned the task reasonably well. I adjusted the training parameters—such as learning rate, batch size, and number of epochs—to achieve stable and acceptable performance given the limited resources. Despite hardware constraints, this setup allowed me to fine-tune GPT-2 effectively and gain practical experience in optimization and model management.

## Save final model

In [ ]:
model.save_pretrained("/content/drive/MyDrive/gpt2_finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/gpt2_finetuned")

Previously, I tried fine-tuning a LLaMA-based chatbot on my dataset, but training was extremely slow and often exceeded Colab Free limits, causing progress loss. I do not have access to Colab Pro, other paid services, or a powerful local GPU, so I switched to **GPT-2**, a smaller model that trains faster and requires less memory. Despite LLaMA initially showing slightly better results, this change allowed me to work effectively with the resources I have and focus on **practical fine-tuning skills, optimization, and checkpoint management**, gaining hands-on experience while still achieving a functional chatbot.